In [ ]:
%pip install -r requirements.txt -q

In [ ]:
# PyTorch support apple silicon
# Ref: https://developer.apple.com/metal/pytorch/
# Check if MPS is available and print the device
import torch
from pprint import pprint

if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")


In [ ]:
# Ref: https://huggingface.co/blog/train-sentence-transformers

from datasets import load_dataset
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    SentenceTransformerModelCardData,
)
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import TripletEvaluator

In [ ]:
# Define constants
run_name = "mpnet-base-all-nli-triplet"
base_model_name = "sentence-transformers/all-MiniLM-L6-v2"
output_model_dir = f"models/{run_name}"


In [ ]:
# 1. Load a model to finetune with 2. (Optional) model card data
model = SentenceTransformer(
  base_model_name,
  model_card_data=SentenceTransformerModelCardData(
        language="en",
        license="apache-2.0",
        model_name="TXOne Networks",
    )
  )


In [ ]:
# 3. Load a dataset to finetune on
dataset = load_dataset("sentence-transformers/all-nli", "triplet")

train_dataset = dataset["train"].select(range(100_000))
eval_dataset = dataset["dev"]
test_dataset = dataset["test"]


In [ ]:
# 4. Define a loss function
loss = MultipleNegativesRankingLoss(model)


In [ ]:
# 5. (Optional) Specify training arguments
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir=output_model_dir,

    # Optional training parameters:
    num_train_epochs=1,
    # If per_device_train_batch_size 8 and you are using 2 GPUs,
    # each GPU will process 8 samples per batch, resulting in a total batch size of 16 across all devices.
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,

    warmup_ratio=0.1,

    fp16=False,  # Set to False if GPU can't handle FP16
    bf16=True,  # Set to True if GPU supports BF16

    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicates

    use_mps_device=True,

    # Optional tracking/debugging parameters:

    eval_strategy="steps",
    eval_steps=100,

    # The checkpoint save strategy to adopt during training.
    # ”no”: No save is done during training.
    # ”epoch”: Save is done at the end of each epoch.
    # ”steps”: Save is done every save_steps.
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,

    logging_steps=100,

    run_name=run_name,  # Used in W&B if `wandb` is installed
)


In [ ]:
# 6. (Optional) Create an evaluator & evaluate the base model
dev_evaluator = TripletEvaluator(
    anchors=eval_dataset["anchor"],
    positives=eval_dataset["positive"],
    negatives=eval_dataset["negative"],
    name="dev",
)
dev_evaluator(model)


In [ ]:
# 7. Create a trainer & train
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=loss,
    evaluator=dev_evaluator,
)
trainer.train()


In [ ]:
# (Optional) Evaluate the trained model on the test set, after training completes
test_evaluator = TripletEvaluator(
    anchors=test_dataset["anchor"],
    positives=test_dataset["positive"],
    negatives=test_dataset["negative"],
    name="test",
)
test_evaluator(model)
